# ML model development and Deployment using azureML (without mlflow)

When you run a script as an Azure Machine Learning job, you need to define the execution context for the job run. One key configuration is the compute target on which the script will be run. This could be the local workstation (in this case the compute instance), or a remote compute target such as the Azure Machine Learning managed compute cluster that is provisioned on-demand.

In this notebook, you'll create a compute cluster and explore compute targets for jobs.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [33]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.6.0
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: c:\users\anshu\appdata\roaming\python\python310\site-packages
Requires: azure-common, azure-core, azure-mgmt-core, azure-storage-blob, azure-storage-file-datalake, azure-storage-file-share, colorama, isodate, jsonschema, marshmallow, msrest, opencensus-ext-azure, pydash, pyjwt, pyyaml, strictyaml, tqdm, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. The resource group name and workspace name are already filled in for you. You only need the subscription ID to complete the command.

To find the necessary parameters, click on the subscription and workspace name at the top right of the Studio. A pane will open on the right.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Copy the subscription ID and replace **YOUR-SUBSCRIPTION-ID** with the value you copied. </p>

In [1]:
# enter details of your AML workspace
subscription_id = "84a5808b-5549-459a-98f2-f102e84fa1bb"
resource_group = "EYMAY"
workspace = "aml-anshu2"

In [2]:
from azure.identity import DefaultAzureCredential,InteractiveBrowserCredential
credential = InteractiveBrowserCredential(tenant_id="13a86542-2185-4187-8e07-7512f5525c55")

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# get a handle to the workspace
ml_client = MLClient(
    credential, subscription_id, resource_group, workspace
)

Class FeatureStoreOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureSetOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class FeatureStoreEntityOperations: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [4]:
from azure.ai.ml.entities import Workspace

In [5]:
try:
    ws = Workspace(name="aml-anshu2",location="centralindia",
                )

    ws = ml_client.workspaces.begin_create(ws)
except:
    print("workspace already exists")


Class ManagedNetwork: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
The deployment request aml-anshu2-7799711 was accepted. ARM deployment URI for reference: 
https://portal.azure.com//#blade/HubsExtension/DeploymentDetailsBlade/overview/id/%2Fsubscriptions%2F84a5808b-5549-459a-98f2-f102e84fa1bb%2FresourceGroups%2FEYMAY%2Fproviders%2FMicrosoft.Resources%2Fdeployments%2Faml-anshu2-7799711


## Create a compute cluster

In many cases, your local compute resources may not be sufficient to process a complex or long-running experiment that needs to process a large volume of data; and you may want to take advantage of the ability to dynamically create and use compute resources in the cloud. Azure Machine Learning supports a range of compute targets, which you can define in your workpace and use to run jobs; paying for the resources only when using them.

You can create a compute cluster in [Azure Machine Learning studio](https://ml.azure.com), by using the Python SDK, or the Azure CLI. The following code cell checks your workspace for the existence of a compute cluster names `aml-cluster`, and if it doesn't exist, creates it.

In [6]:
from azure.ai.ml.entities import AmlCompute

# Name assigned to the compute cluster
cpu_compute_target = "aml-cluster"

try:
    # let's see if the compute target already exists
    cpu_cluster = ml_client.compute.get(cpu_compute_target)
    print(
        f"You already have a cluster named {cpu_compute_target}, we'll reuse it as is."
    )

except Exception:
    print("Creating a new cpu compute target...")

    # Let's create the Azure ML compute object with the intended parameters
    cpu_cluster = AmlCompute(
        name=cpu_compute_target,
        # Azure ML Compute is the on-demand VM service
        type="amlcompute",
        # VM Family
        size="STANDARD_DS11_V2",
        # Minimum running nodes when there is no job running
        min_instances=0,
        # Nodes in cluster
        max_instances=2,
        # How many seconds will the node running after the job termination
        idle_time_before_scale_down=120,
        # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
        tier="Dedicated",
    )

    # Now, we pass the object to MLClient's create_or_update method
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)


Creating a new cpu compute target...


After you've created a compute cluster, you can only change the configuration for:

- `min_instances`: Minimum number of nodes
- `max_instances`: Maximum number of nodes
- `idle_time_before_scale_down`: Idle time before scale down

Currently, your compute cluster `aml-cluster` can only scale do a maximum of one node. Let's change that to two, to allow for parallel compute.

In [7]:
from azure.ai.ml.entities import AmlCompute

cluster_scale = AmlCompute(
    name="aml-cluster",
    max_instances=2,
)
ml_client.begin_create_or_update(cluster_scale)

When the compute cluster is updated, you can verify its configuration by printing its attributes.

In [8]:
cpu_cluster = ml_client.compute.get("aml-cluster")

print (
        f"AMLCompute with name {cpu_cluster.name} has a maximum of {cpu_cluster.max_instances} nodes"
    )

AMLCompute with name aml-cluster has a maximum of 2 nodes


## Create a script to train a model

To train a model, you'll first create the **diabetes_training.py** script in the **src** folder. The script uses the **diabetes.csv** file in the same folder as the training data.

In [9]:
import os
os.makedirs("src",exist_ok=True)

In [10]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)



# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

import os
import joblib
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/model.pkl')


Overwriting src/diabetes-training.py


## Run a job on a compute cluster

Now, you're ready to run the job on the compute cluster you created.

> **Note**:
> The job will take some time to start as the compute cluster will need to scale from zero to one node. Once the compute cluster is ready, the script will be run. When the job has finished, the compute cluster will scale back down to zero nodes. You can review the compute cluster's status in the **Compute** page.

In [11]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-cluster",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.53 MBs): 100%|##########| 530055/530055 [00:00<00:00, 1589044.72it/s]




Monitor your job at https://ml.azure.com/runs/helpful_malanga_jhdrp6kx27?wsid=/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourcegroups/EYMAY/workspaces/aml-anshu2&tid=13a86542-2185-4187-8e07-7512f5525c55


<h4 style="background-color:Tomato;">wait till the job runs successfully, </h4>

In [12]:
print(returned_job.display_name,returned_job,returned_job.name)

diabetes-train-cluster type: command
outputs:
  default:
    mode: rw_mount
    type: uri_folder
    path: azureml://datastores/workspaceartifactstore/ExperimentRun/dcid.helpful_malanga_jhdrp6kx27
environment: azureml:AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest
resources:
  instance_count: 1
  shm_size: 2g
services:
  Tracking:
    endpoint: azureml://centralindia.api.azureml.ms/mlflow/v1.0/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspaces/aml-anshu2?
    type: Tracking
  Studio:
    endpoint: https://ml.azure.com/runs/helpful_malanga_jhdrp6kx27?wsid=/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourcegroups/EYMAY/workspaces/aml-anshu2&tid=13a86542-2185-4187-8e07-7512f5525c55
    type: Studio
compute: azureml:aml-cluster
component:
  name: helpful_malanga_jhdrp6kx27
  display_name: diabetes-train-cluster
  type: command
  outputs:
    default:
      type: uri_folder
      mode: rw_mount
  command:


    A path on your local computer	                    mlflow-model/model.pkl
    A path on an Azure Machine Learning Datastore	    azureml://datastores/<datastore-name>/paths/<path_on_datastore>
    A path from an Azure Machine Learning job	        azureml://jobs/<job-name>/outputs/<output-name>/paths/<path-to-model-relative-to-the-named-output-location>
    A path from an MLflow job	                        runs:/<run-id>/<path-to-model-relative-to-the-root-of-the-artifact-location>
    A path from a Model Asset in Azure Machine Learning Workspace	azureml:<model-name>:<version>
    A path from a Model Asset in Azure Machine Learning Registry	azureml://registries/<registry-name>/models/<model-name>/versions/<version>

In [13]:
# register model
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
path = f"azureml://jobs/{returned_job.name}/outputs/artifacts/paths/outputs/model.pkl"

file_model = Model(path=path,type=AssetTypes.CUSTOM_MODEL,
                   name="diabetes_model")

out = ml_client.models.create_or_update(file_model)
out.path

'azureml://subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EYMAY/workspaces/aml-anshu2/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.helpful_malanga_jhdrp6kx27/outputs/model.pkl'

In [14]:
out.name

'diabetes_model'

In [15]:
out.version

'1'

# Deployment: Real time online deployment

In [26]:
from azure.ai.ml.entities import ManagedOnlineEndpoint
import datetime

ep_name = "endpoint-diabetes" 
ep = ManagedOnlineEndpoint(name = ep_name,
                           auth_mode="key")
ep_result = ml_client.online_endpoints.begin_create_or_update(ep).result()

In [29]:
ep = ml_client.online_endpoints.get(name="endpoint-diabetes")

In [30]:
import os
os.makedirs("score",exist_ok=True)

In [47]:
%%writefile score/score.py

import os
import joblib
import json 
import logging
import numpy
def init():
    global model
    path = os.path.join(os.getenv("AZUREML_MODEL_DIR"),"model.pkl") # AZUREML_MODEL_DIR = ./azureml-models/MODEL_NAME/VERSION
    logging.info(path)
    try:
        model = joblib.load(path)
    except Exception as e:
        logging.info("Error occurred")
        logging.info(e)
        logging.info("error end")
    logging.info("initialization completed")

def run(raw_data):
    logging.info("model: request received")
    data = json.loads(raw_data)["data"]
    data = numpy.array(data)
    result = model.predict(data)
    logging.info("request procsesed")
    return result.tolist()

Overwriting score/score.py


In [48]:
%%writefile score/conda.yml
name: model-env
channels:
  - conda-forge
dependencies:
  - python=3.7
  - numpy=1.21.2
  - pip=21.2.4
  - scikit-learn=0.24.2
  - scipy=1.7.1
  - numpy
  - pandas
  - pip:
    - azureml-defaults==1.47.0
    - applicationinsights
    - inference-schema[numpy-support]==1.5
    - joblib==1.0.1

Overwriting score/conda.yml


In [49]:
# deployment configuration
from azure.ai.ml.entities import Model,ManagedOnlineDeployment,CodeConfiguration
from azure.ai.ml.constants import AssetTypes

# configure an environment
from azure.ai.ml.entities import Environment
env = Environment(
    conda_file="score/conda.yml",
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210727.v1",
)

# configure an inference configuration with a scoring script
from azure.ai.ml.entities import CodeConfiguration
code_config = CodeConfiguration(
        code="score", scoring_script="score.py"
    )

model = ml_client.models.get(name=out.name, version=out.version)

# define an online deployment
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=ep_name,
    model=model,
    instance_type="Standard_DS1_v2",
    instance_count=1,
    environment=env,
    code_configuration=code_config
)

In [46]:
blue_deploy_result = ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/en-us/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint endpoint-diabetes exists
Uploading score (0.0 MBs): 100%|##########| 1017/1017 [00:00<00:00, 3203.99it/s]


data_collector is not a known attribute of class <class 'azure.ai.ml._restclient.v2022_02_01_preview.models._models_py3.ManagedOnlineDeployment'> and will be ignored


..............................

HttpResponseError: (None) ResourceNotReady: User container has crashed or terminated: Liveness probe failed: Get http://10.66.0.5:5001/: net/http: request canceled (Client.Timeout exceeded while awaiting headers). Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Code: None
Message: ResourceNotReady: User container has crashed or terminated: Liveness probe failed: Get http://10.66.0.5:5001/: net/http: request canceled (Client.Timeout exceeded while awaiting headers). Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready

In [ ]:
# blue deployment takes 100 traffic
ep.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(ep)

#### Check the status of the endpoint
You can check the status of the endpoint to see whether the model was deployed without error:


In [ ]:

# return an object that contains metadata for the endpoint
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

# print a selection of the endpoint's metadata
print(
    f"Name: {endpoint.name}\nStatus: {endpoint.provisioning_state}\nDescription: {endpoint.description}"
)


In [ ]:
# existing traffic details
print(endpoint.traffic)

### Inference

In [ ]:
os.makedirs("testing",exist_ok=True)

In [ ]:
%%writefile sample-request.json
{"data": [
 [9,103,78,25,304,29.58219193,1.282869847,43], 
 [7,115,47,52,35,41.51152348,0.079018568,23]
]}

In [35]:
# using json file to send a request using ml client
# test the blue deployment with some sample data
ml_client.online_endpoints.invoke(
    endpoint_name=ep_name,
    deployment_name="blue",
    request_file="testing/sample-request.json",
)

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://endpoint-diabetes2.westus.inference.ml.azure.com/score', 'openapi_uri': 'https://endpoint-diabetes2.westus.inference.ml.azure.com/swagger.json', 'name': 'endpoint-diabetes2', 'description': None, 'tags': {}, 'properties': {'azureml.onlineendpointid': '/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourcegroups/eymay/providers/microsoft.machinelearningservices/workspaces/aml-anshu2/onlineendpoints/endpoint-diabetes2', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/providers/Microsoft.MachineLearningServices/locations/westus/mfeOperationsStatus/oe:c8702cfe-ee2e-4ded-a217-a1310873a604:c712df0f-58c7-4277-ac31-b80dc00119cd?api-version=2022-02-01-preview'}, 'print_as_yaml': True, 'id': '/subscriptions/84a5808b-5549-459a-98f2-f102e84fa1bb/resourceGroups/EYMAY/providers/Microsoft.MachineLearningServices/workspac

In [ ]:
print("Scoring URI:", ep.scoring_uri)

In [ ]:
# sample testing via rest API

In [ ]:
# delete the endpoint and deployment
ml_client.online_endpoints.begin_delete(name=ep_name)